## Exercise some challenging cadences

This is based on a suggestion by Paul Erlich in a Facebook post on 9/2/23 to test my automatic just intonation.

<blockquote>
Curious how your algorithm would handle the basic I-IV-ii-V-I and I-vi-ii-V-I progressions arranged to maximize common tones between consecutive chords. Are there any pitch shifts / glides greater than 6 cents? This is a good set of test cases for any adaptive JI algorithm.
</blockquote>

This is presenting several challenges together. 

- The first is just to get the algorithm to come up with a working just intonation version of these two cadences. I accomplished that.
- The second is to maximize common tones. I don't give any value in my algoritm to that characteristic, so I don't expect to do well on that. It did a great job on that, but had to slide the D by 22 cents.
- The third is to ensure no pitch changes or glides greater than 6 cents. Again, I don't have any incentives in the algorithm to ensure that. It didn't. The D had to move 22 cents. The only way to get that done is to corrupt the cent values of some notes so to minimize the movement of the D. That sound like temperament, and I'm trying to get away from that.

Mine is only a vertical just intonation, and he is asking for horizontal awareness. My first and only effort to implement horizontal just intonation was in the implementation of slides between two cent versions of the same consecutive note. In that case I slide from one to the other using a cubic polynomial.

In [896]:
import sys
sys.path.insert(0, '../Diamond_Music') # you must have already obtained the Diamond_Music repo from github and put it in the right dir.
import diamond_music_utils as dmu

import adaptive_tuning_util as atu
import numpy as np
rng = np.random.default_rng()
import os
from importlib import reload
import music21 as m21
import logging
from itertools import count, combinations, permutations
import matplotlib.pyplot as plt
import pprint as pp
# some constants 

flats = True # set this to False if the key uses sharps. It will later get set based on reading the key signature from the corpus.
keys = atu.set_accidentals(flats)
CSD_FILE = 'ball9.csd'
JUPYTER_LOG = 'erlich_cadence.log'
dmu.start_logger(JUPYTER_LOG, log_level = 'info')
CS_LOGNAME = 'slide_tuning.log'
MIDI_DIR = '.' 
NUMPY_DIR = 'eval/numpy_chorales'
WAVE_DIR = '../../../Music/sflib'
m21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore') # required for finding mscore
reload(dmu)
reload(atu)

<module 'adaptive_tuning_util' from '/home/prent/Dropbox/Tutorials/TonicNet/adaptive_tuning_util.py'>

In [897]:
voice_time = atu.init_voice_time()
pp.pprint(voice_time, sort_dicts=False)

{'fing1': {'full_name': 'finger piano 1',
           'start': 0,
           'csound_voice': 1,
           'time_tracker_number': 0,
           'volume_factor': 0,
           'min_oct': 2,
           'max_oct': 7},
 'fing2': {'full_name': 'finger piano 2',
           'start': 0,
           'csound_voice': 1,
           'time_tracker_number': 1,
           'volume_factor': 0,
           'min_oct': 2,
           'max_oct': 7},
 'fing3': {'full_name': 'finger piano 3',
           'start': 0,
           'csound_voice': 1,
           'time_tracker_number': 2,
           'volume_factor': 0,
           'min_oct': 2,
           'max_oct': 7},
 'bfin1': {'full_name': 'bass finger piano 1',
           'start': 0,
           'csound_voice': 24,
           'time_tracker_number': 3,
           'volume_factor': 1,
           'min_oct': 1,
           'max_oct': 5},
 'fing4': {'full_name': 'finger piano 4',
           'start': 0,
           'csound_voice': 1,
           'time_tracker_number': 4,
      

In [898]:
def tune_chorale(chorale, keys, top_notes, ratio_factor = 1, tolerance = 1):   
    logging.info(f'{ratio_factor = }')  
    dist_factor = 1 / ratio_factor
    stop_when = 10 # stop the roll process if you get a score this low
    min_score_perm = 10 # this is the minimum score required to accept a chord from improve_chord_rolls. If it's higher than that, we must send it to try_permutations to try different permutations. If you set this very low, it always goes to try_permutations
    limit_max = 31 # What is the limit of the tonality diamond. I have only had success with 31-limit
    original_12 = np.arange(0, 1200, 100) # I tried carefully choosing the initial cent values by hand, but it had bad effects.
    tonal_diamond = np.array(atu.build_tonal_diamond(limit_max)) # load the 213 ratios and their cent values and numerator and denominators. Add one more for the ratio 2:1 to make 214
    logging.info(f'{tonal_diamond.shape = }')  # (213, 3)
    # here is where we assign anchor notes that the algorithm transpose whole chords to maintain these locations, starting with the first one
   
    logging.info(f'{top_notes = }')
    logging.info(f'{keys[top_notes[0] % 12] = }')
    chorale_in_cents = atu.midi_to_notes_octaves_trimmed(chorale, top_notes, tonal_diamond, ratio_factor = ratio_factor, dist_factor = dist_factor, stop_when = stop_when, flats = flats, min_score_perm = min_score_perm, original_12 = original_12, range = 6, tolerance = tolerance)
    return(chorale_in_cents, tonal_diamond, ratio_factor, top_notes)
# end of tune_chorale

In [899]:
# voice_names = np.array(['flut1', 'oboe1', 'frnh1', 'basn1', 'clar1', 'oboe2', 'frnh2', 'basn2']) 
voice_names = np.array(['flut1', 'clar1', 'oboe1', 'oboe2', 'frnh1', 'frnh2', 'basn1', 'basn2'])
for inx in np.arange(8):
    print(voice_names[inx])

flut1
clar1
oboe1
oboe2
frnh1
frnh2
basn1
basn2


In [900]:
def trim_csound(version, duration, trim = True):
    logging.info(f'{version = }')
    result = 0  
    if trim:
        os.system(f"sed -i 's/replaceme/{duration}/' ball9c.csd") # change the duration field in the csound convolution operation to the length in seconds of the wave file.
        result = os.system(f'sh trim.sh ball9 {version}') # carry out the tasks in trim.sh, which include the convolution csound operation, and a sox trimming of the resulting wave file. 
        os.system(f"sed -i 's/{duration}/replaceme/' ball9c.csd") # change it back to what it was previously after you finish the trim.sh operation
    else: print(f'Please note that you set {trim = } which means it will not be convolved.')   
    return result

def show_chorale(s):
    # s.show("musicxml.png")
    s.show()

In [901]:
def woodwinds_part(chorale_in_cents_slides, glides, stored_gliss, repeats, voice_names, voice_time, tpq):

      print(f'in woodwinds_part. {chorale_in_cents_slides.shape = }, {glides.shape = }, {stored_gliss.shape = }, {repeats = }') 
      print(f'{voice_names = } ')
      voices = voice_names.shape[0] # if you want it to last twice as long, pretend there are twice as many voices: voice_names.shape[0] * 2
      chorale_in_cents_slides = np.repeat(chorale_in_cents_slides, repeats, axis = 1) # make each note repeats making it n times as long on each note.
      glides = np.repeat(glides, repeats, axis = 1)
      logging.debug(f'after repeating each note {repeats = }: {glides.shape = }')
      chorale_in_cents_slides = np.repeat(chorale_in_cents_slides, voices // 4, axis = 0) # make the proper number of voices so each voice gets one track. 
      glides = np.repeat(glides, voices // 4, axis = 0) # to to glides what you just did to chorale_in_cents_slides
      logging.info(f'after doubling voices: {chorale_in_cents_slides.shape = }, {glides.shape = }')  # (4, 16, 2), (4, 16)

      notes = chorale_in_cents_slides[:,:,0] # the 0th feature is the note # (4, 256)
      octaves =  chorale_in_cents_slides[:,:,1] # the 0th feature is the note # (4, 256)
      
      
      logging.info(f'{np.average(octaves) = }')
      velocity = np.zeros_like(notes)   
      
      velocity[[0,1],:] = 55 # 13, 14 flute, clarinet
      velocity[[2,3],:] = 55 # 15, 15 oboes
      velocity[[4,5],:] = 55 # 16, 16 french horns
      velocity[[6,7],:] = 55 # 12 bassoons
      print(f'{velocity = }')
      # this is not the order of the notes I was expecting. I wanted flute & clarinet on soprano
      # oboes on alto, french horns on tenor, bassoons on bass. How can I achieve that?
      # well, here is your answer. They are following this order from the voice_names array
            #                    0         1        2        3        4        5        6        7
            #                   0        1      2        3       4         5        6        7
      # voice_names = array(['flut1', 'oboe1', 'frnh1', 'basn1', 'clar1', 'oboe2', 'frnh2', 'basn2'] # what I had at first
      # voice_names = array(['flut1', 'clar1', 'oboe1', 'oboe2', 'fnrh1', 'fnrh2', 'basn1', 'basn2'] # what I thought would fix it
      # voice_names = array(['flut1', 'clar1', 'oboe1', 'oboe2', 'fnrh1', 'fnrh2', 'basn1', 'basn2']

      upsample = np.zeros_like(notes) # start with no upsample
      envelope = np.ones_like(notes) # start with 1 as the envelope
      # I need to replace atu.add_features_glides with something simpler:
      logging.info(f'{notes.shape = }, {octaves.shape = }, {glides.shape = }, {velocity.shape = }') # notes.shape = (8, 16), octaves.shape = (8, 16), glides.shape = (8, 16) - 8 voices, 16 notes
      notes_features_6 = np.stack((notes, octaves, glides, upsample, envelope, velocity), axis = 0) 
      logging.info(f'{np.average(notes_features_6[1]) = }') # 5.140625

      notes_features_6[1] -= 1 # lower the octaves by 1
      logging.info(f'{np.average(notes_features_6[1]) = }') # 4.140625
      logging.info(f'{notes_features_6.shape = }')
      volume_array = np.ones(notes_features_6.shape[2], dtype = int) * 60
      notes_features_15 = dmu.piano_roll_to_notes_features(notes_features_6, volume_array, voice_names, tpq, voice_time)
      #  1, dur,      hol,         vel,            note,         octv,       voice,      stereo
     
      
      # notes_features_15 = atu.clip_note_features(notes_features_15, voice_time) # make sure the octaves are in range and the volume adjusted per the voice_time dictionary
      logging.info(f'{notes_features_15.shape = }')
      
      return notes_features_15


In [902]:
reload(atu)
reload(dmu)
dmu.start_logger(JUPYTER_LOG, log_level = 'info')
# I need to initialize the voice_time dictionary with the start times of each voice.
voice_time = atu.init_voice_time()
tempo = 100
limit = 1000 
# volume_function = np.full((6, 9), 2, dtype = int)
repeats = 2
best_tops = np.array([[5, 4, 7, 0], [498, 386, 702, 0]] )
top_notes = np.array([[5],[498]])

for ratio_factor in ([1.5]):
    chorale, root, mode, s, pic_cl, pcu = atu.read_from_midi('I-IV-ii-V-I.mid', quantizer = 1)
    print(f'{chorale.shape = }, {keys[root] = }, {mode = }')
    if mode == 'minor':
            if root in ([2,7,0,5,10,3]): # minor keys notated with flats d, g, c, f, bb, eb
                keys = atu.set_accidentals(True) # True = flats False = sharps
            else: keys = atu.set_accidentals(False) 
    else:    
        if root in ([7,2,9,4,11,6]): # major keys notated with sharps" G D A E B, F#
            keys = atu.set_accidentals(False) # True = flats False = sharps
        else: keys = atu.set_accidentals(True) 
    print(f'{top_notes = }, {ratio_factor = }')
    chorale_in_cents, tonal_diamond, ratio_factor, top_notes = tune_chorale(chorale, keys, top_notes, ratio_factor = ratio_factor, tolerance = 1)            
    print(f'{chorale_in_cents.shape = }, {top_notes = }, {tonal_diamond.shape = }, {ratio_factor = }')
    if atu.mismatch_check(chorale_in_cents, chorale):
        print(f'Warning: This mismatch is a serious problem. The algorithm tuned a note to a different midi note than the composer intended.')

    atu.print_interval_cent_report(chorale_in_cents, chorale, top_notes, tonal_diamond, keys, ratio_factor, tolerance = 1)

    chorale_in_cents_slides, glides, stored_gliss, t_num = atu.build_glides_array(chorale_in_cents, keys)

    notes_features_15 = woodwinds_part(chorale_in_cents_slides, glides, stored_gliss, repeats, voice_names, voice_time, 1)
    

    notes_features_final, voice_time = dmu.fix_start_times(notes_features_15, voice_time)
    logging.info(f'{notes_features_final[:, 5] = }') # notes_features_final[:, 5] = array([6., 6., all good

    logging.info(f'about to update_gliss_table with {stored_gliss.shape = }')
    tables = dmu.update_gliss_table(stored_gliss, stored_gliss.shape[0])
    logging.info(f'back from update_gliss_table with {stored_gliss.shape = }, {tables = }')
    logging.info(f'About to call send_to_csound_file. {notes_features_final.shape = }')

    notes_features = dmu.send_to_csound_file(notes_features_final, voice_time, CSD_FILE, tempos = 't0 ' + str(tempo), limit = limit, tempo = tempo, print_only = 90) 
    logging.info(f'{notes_features.shape = }')  # notes_features.shape = (0,) That doesn't look right.
    !csound new_output.csd -Onew_output.log
    version = '_' + str(ratio_factor)
    duration = 60
    
    result_of_call = trim_csound(version, duration, trim = True)
    if result_of_call != 0: print(f'possible failure. {result_of_call = }')


chorale.shape = (4, 16), keys[root] = 'C♮', mode = 'major'
top_notes = array([[  5],
       [498]]), ratio_factor = 1.5
in midi_to_notes_octaves. octave.shape = (4, 16)
chorale_in_cents.shape = (4, 16, 2), top_notes = array([[  5],
       [498]]), tonal_diamond.shape = (214, 3), ratio_factor = 1.5
report the chords used, with chord scores
#		names of the notes	cents of notes		intervals between notes, the cents and ratios of the intervals		chord score
0 		['C♮', 'G♮', 'E♮', 'C♮']	[  0 702 386   0]	(0, 1, 702, '3/2') (0, 2, 386, '5/4') (0, 3, 0, '1') (1, 2, 316, '6/5') (1, 3, 702, '3/2') (2, 3, 386, '5/4')		39.0
1 		['C♮', 'A♮', 'F♮', 'F♮']	[  0 884 498 498]	(0, 1, 884, '5/3') (0, 2, 498, '4/3') (0, 3, 498, '4/3') (1, 2, 386, '5/4') (1, 3, 386, '5/4') (2, 3, 0, '1')		40.0
2 		['D♮', 'A♮', 'F♮', 'D♮']	[182 884 498 182]	(0, 1, 702, '3/2') (0, 2, 316, '6/5') (0, 3, 0, '1') (1, 2, 386, '5/4') (1, 3, 702, '3/2') (2, 3, 316, '6/5')		41.0
3 		['D♮', 'B♮', 'G♮', 'G♮']	[ 200 1084  698  698]	(0, 1

In [903]:
best_tops = np.array([[5, 4, 7, 0], [498, 386, 702, 0]] )
print(f'{best_tops.T[0] = }')
for inx in np.arange(best_tops.shape[1]):
    top_notes = best_tops[:,inx].reshape(2,1)
    print(f'{top_notes = }')
    print(f'{str(int(top_notes[0]))}')

best_tops.T[0] = array([  5, 498])
top_notes = array([[  5],
       [498]])
5
top_notes = array([[  4],
       [386]])
4
top_notes = array([[  7],
       [702]])
7
top_notes = array([[0],
       [0]])
0
